<a href="https://colab.research.google.com/github/jankovicsandras/plpgsql_bm25/blob/main/Postgres_hybrid_search_RRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Postgres hybrid search with Reciprocal Rank Fusion

https://github.com/jankovicsandras/plpgsql_bm25

https://github.com/pgvector/pgvector

## installing PostgreSQL

In [1]:
! sudo apt install gnupg2 wget nano
! sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
! curl -fsSL https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo gpg --dearmor -o /etc/apt/trusted.gpg.d/postgresql.gpg
! sudo apt update
! sudo apt install postgresql-16 postgresql-contrib-16 postgresql-server-dev-16
! service postgresql start
! sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
gnupg2 is already the newest version (2.2.27-3ubuntu2.1).
Suggested packages:
  hunspell
The following NEW packages will be installed:
  nano
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 280 kB of archives.
After this operation, 881 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 nano amd64 6.2-1ubuntu0.1 [280 kB]
Fetched 280 kB in 1s (364 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-p

## init psycopg2 helper

In [2]:
! pip install psycopg2

import psycopg2


def msq( t, verbose=False ) :
  res = []
  with psycopg2.connect("dbname=postgres user=root") as conn:
    with conn.cursor() as cur:
      try :
        cur.execute(t)
        res = cur.fetchall()
        if verbose :
          for cdesc in cur.description :
            print(cdesc[0],'|',end='')
          print('')
          for r in res:
            for c in r:
              print(c,'|',end='')
            print('')
      except Exception as ex :
        print('!!!! msq() ERROR ',ex,'|',t,'|')
  return res


def iscorrectintopk(correct, res) :
  cr = correct.replace('"','\'')
  if len(res) < 1 : return 0
  for cell in res[0] :
    if str(cell).replace('"','\'') == cr : return 2
  for ri in range(1,len(res)) :
    for cell in res[ri] :
      if str(cell).replace('"','\'') == cr : return 1
  return 0


## installing pgvector

In [3]:
! git clone --branch v0.8.0 https://github.com/pgvector/pgvector.git
! cd pgvector && make
! cd pgvector && sudo make install
msq('CREATE EXTENSION vector;')

Cloning into 'pgvector'...
remote: Enumerating objects: 11630, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 11630 (delta 209), reused 236 (delta 184), pack-reused 11319 (from 1)
Receiving objects: 100% (11630/11630), 1.80 MiB | 10.47 MiB/s, done.
Resolving deltas: 100% (8691/8691), done.
Note: switching to '2627c5ff775ae6d7aef0c430121ccf857842d2f2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclarat

[]

## plpgsql_bm25

In [4]:
# downloading from https://github.com/jankovicsandras/plpgsql_bm25
! wget -nc https://raw.githubusercontent.com/jankovicsandras/plpgsql_bm25/refs/heads/main/plpgsql_bm25.sql

# execute to load the functions
! psql postgresql://@/postgres -f /content/plpgsql_bm25.sql

# downloading from https://github.com/jankovicsandras/plpgsql_bm25
! wget -nc https://raw.githubusercontent.com/jankovicsandras/plpgsql_bm25/refs/heads/main/plpgsql_bm25rrf.sql

# execute to load the functions
! psql postgresql://@/postgres -f /content/plpgsql_bm25rrf.sql

--2024-12-05 21:04:06--  https://raw.githubusercontent.com/jankovicsandras/plpgsql_bm25/refs/heads/main/plpgsql_bm25.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13696 (13K) [text/plain]
Saving to: ‘plpgsql_bm25.sql’

plpgsql_bm25.sql    100%[===================>]  13.38K  --.-KB/s    in 0s      

2024-12-05 21:04:06 (73.7 MB/s) - ‘plpgsql_bm25.sql’ saved [13696/13696]

psql:/content/plpgsql_bm25.sql:23: NOTICE:  function bm25simpletokenize() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:/content/plpgsql_bm25.sql:44: NOTICE:  function count_words_in_array() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:/content/plpgsql_bm25.sql:65: NOTICE:  function get_word_docs_count() does not exist, skipping
DROP FUNCTION
CREATE FUNCTIO

In [5]:
! pip install sentence-transformers

## Test datasets


In [6]:

! wget -nc https://zav-public.s3.amazonaws.com/inpars/msmarco_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/robust04_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/nq_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/trec_covid_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/fiqa_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/dbpedia_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/scidocs_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/scifacts_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/arguana_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/bioasq_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/climate_fever_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/cqadupstack_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/fever_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/hotpotqa_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/nfcorpus_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/quora_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/signal_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/touche_synthetic_queries_100k.jsonl
! wget -nc https://zav-public.s3.amazonaws.com/inpars/trec_news_synthetic_queries_100k.jsonl


--2024-12-05 21:04:15--  https://zav-public.s3.amazonaws.com/inpars/msmarco_synthetic_queries_100k.jsonl
Resolving zav-public.s3.amazonaws.com (zav-public.s3.amazonaws.com)... 52.219.72.177, 52.219.169.131, 3.5.134.86, ...
Connecting to zav-public.s3.amazonaws.com (zav-public.s3.amazonaws.com)|52.219.72.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56420140 (54M) [application/x-www-form-urlencoded]
Saving to: ‘msmarco_synthetic_queries_100k.jsonl’

msmarco_synthetic_q 100%[===================>]  53.81M  19.7MB/s    in 2.7s    

2024-12-05 21:04:19 (19.7 MB/s) - ‘msmarco_synthetic_queries_100k.jsonl’ saved [56420140/56420140]

--2024-12-05 21:04:19--  https://zav-public.s3.amazonaws.com/inpars/robust04_synthetic_queries_100k.jsonl
Resolving zav-public.s3.amazonaws.com (zav-public.s3.amazonaws.com)... 52.219.72.177, 52.219.169.131, 3.5.134.86, ...
Connecting to zav-public.s3.amazonaws.com (zav-public.s3.amazonaws.com)|52.219.72.177|:443... connected.
HTTP 

In [7]:
import json, random
from sentence_transformers import SentenceTransformer

verbose = False
k = 99999
slimit = 99999
samplenum = 200
embeddermodel = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
vectordim = 384
datasetnames = ['msmarco', 'robust04', 'nq', 'trec_covid', 'fiqa', 'dbpedia', 'scidocs', 'scifacts', 'arguana', 'bioasq', 'climate_fever', 'cqadupstack', 'fever', 'hotpotqa', 'nfcorpus', 'quora', 'signal', 'touche', 'trec_news']
fnpostfix = '_synthetic_queries_100k.jsonl'
columnname = 'doc_text'
idcolumnname = 'id'
embeddingcolumnname = 'embedding'
algo = 'plus'

# data and question containers
alldata = {}
questions = {}
questionsolutions = {}
embeddedquestions = {}

for dsname in datasetnames : #[:2]
  print('\n---- ',dsname)
  # load
  with open( dsname+fnpostfix ) as f:
    alldata[dsname] = [json.loads(line) for line in f]
  # shuffle and truncate to samplenum
  random.shuffle(alldata[dsname])
  alldata[dsname] = alldata[dsname][:samplenum]
  # display
  #for i in range(0,10) :
  #  print(alldata[dsname][i])

  # doc_text embeddings
  embeddedcorpus = embeddermodel.encode( [ thisdoc[columnname] for thisdoc in alldata[dsname] ] )
  for i, item in enumerate(embeddedcorpus) :
    alldata[dsname][i]['embedding'] = embeddedcorpus[i]

  # export to CSV for Postgres
  csvfilename = dsname+'_items.csv'
  with open(csvfilename,'w+') as f:
    f.write( 'id;'+columnname+'\n' )
    for i in range( 0, len(alldata[dsname]) ) :
      f.write( str(i+1)+';"' + alldata[dsname][i][columnname].replace('"','\'')+'";'+ json.dumps( alldata[dsname][i]['embedding'].tolist() ) + '\n')
  print( csvfilename, 'is created' )

  # suffling again and creating questions and questionsolutions
  questions[dsname] = []
  questionsolutions[dsname] = []
  random.shuffle(alldata[dsname])
  for i in range( 0, len(alldata[dsname]) ) :
    questions[dsname].append( alldata[dsname][i]['question'] )
    questionsolutions[dsname].append( alldata[dsname][i][columnname] )
  # question embeddings
  embeddedquestions[dsname] = embeddermodel.encode( questions[dsname] )

  # Postgres creating items table by importing from CSV
  tablename = dsname + '_items_table'
  msq('DROP TABLE IF EXISTS '+tablename+' CASCADE;')
  msq('CREATE TABLE '+tablename+' (id SERIAL, '+columnname+' TEXT, embedding vector('+str(vectordim)+'));')
  msq('COPY '+tablename+' FROM \'/content/'+csvfilename+'\' DELIMITER \';\' CSV HEADER;')
  msq('SELECT * FROM '+tablename+';')


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


----  msmarco
msmarco_items.csv is created
!!!! msq() ERROR  no results to fetch | DROP TABLE IF EXISTS msmarco_items_table CASCADE; |
!!!! msq() ERROR  no results to fetch | CREATE TABLE msmarco_items_table (id SERIAL, doc_text TEXT, embedding vector(384)); |
!!!! msq() ERROR  no results to fetch | COPY msmarco_items_table FROM '/content/msmarco_items.csv' DELIMITER ';' CSV HEADER; |

----  robust04
robust04_items.csv is created
!!!! msq() ERROR  no results to fetch | DROP TABLE IF EXISTS robust04_items_table CASCADE; |
!!!! msq() ERROR  no results to fetch | CREATE TABLE robust04_items_table (id SERIAL, doc_text TEXT, embedding vector(384)); |
!!!! msq() ERROR  no results to fetch | COPY robust04_items_table FROM '/content/robust04_items.csv' DELIMITER ';' CSV HEADER; |

----  nq
nq_items.csv is created
!!!! msq() ERROR  no results to fetch | DROP TABLE IF EXISTS nq_items_table CASCADE; |
!!!! msq() ERROR  no results to fetch | CREATE TABLE nq_items_table (id SERIAL, doc_text TEXT, 

## running the questions


In [8]:
import json, time, math

stats = {
  'total': {
    'questions':0,
    'plpgsql_bm25_speed total':0,
    'plpgsql_bm25_correct total':0,
    'pgvector_speed total':0,
    'pgvector_correct total':0,
    'python_rrf_speed total':0,
    'python_rrf_correct total':0,
    'bm25rrf_speed total':0,
    'bm25rrf_correct total':0
  },
  'diffs':[]
}
stats2 = {}

for dsname in datasetnames : # [:2]
  print('\n----',dsname,'----------------------------------------------------\n')
  # table and file names
  tablename = dsname + '_items_table'

  stats[dsname] = {
    'plpgsql_bm25_speed':[],
    'plpgsql_bm25_correct':[],
    'pgvector_speed':[],
    'pgvector_correct':[],
    'python_rrf_speed':[],
    'python_rrf_correct':[],
    'bm25rrf_speed':[],
    'bm25rrf_correct':[]
  }

  # Postgres has already "items" table, tokenization is built-in
  msq( 'SELECT bm25createindex( \''+tablename+'\', \''+columnname+'\', \''+algo+'\' );', verbose )

  # Running the questions
  for qi,q in enumerate(questions[dsname]) :
    # print question
    if verbose :
      print('\n----Question',qi,':',q)
      if questionsolutions[dsname] and qi<len(questionsolutions[dsname]) : print('Solution:',questionsolutions[dsname][qi])
    stats['total']['questions'] += 1

    # plpgsql_bm25 BM25 search
    if verbose : print('---- plpgsql_bm25')
    starttime = time.time()
    mres = msq( 'SELECT * FROM bm25topk( \''+tablename+'\', \''+columnname+'\', \''+q.replace("'","\'\'")+'\', '+str(k)+', \''+algo +'\' );', verbose )
    stats[dsname]['plpgsql_bm25_speed'].append( (time.time()-starttime) )

    # pgvector search
    if verbose : print('---- pgvector')
    starttime = time.time()
    mres2 = msq( 'SELECT * FROM '+tablename+' ORDER BY embedding <-> \''+json.dumps( embeddedquestions[dsname][qi].tolist() )+'\' LIMIT '+str(k)+';', verbose )
    stats[dsname]['pgvector_speed'].append( (time.time()-starttime) )

    # Reciprocal Rank Fusion https://medium.com/@devalshah1619/mathematical-intuition-behind-reciprocal-rank-fusion-rrf-explained-in-2-mins-002df0cc5e2a
    # rrfscore =  1 / (60+rank1) + 1 / (60+rank2)
    starttime = time.time()
    mres3 = []
    bm25rankeddocs = [ r[2] for r in mres ]
    for ri, r in enumerate(mres2) :
      rrfscore = 1 / ( ri + 61 )
      if r[1] in bm25rankeddocs :
        rrfscore += 1 / ( bm25rankeddocs.index(r[1]) + 61 )
      mres3.append( [ r[0], rrfscore, r[1] ] )
    mres3.sort(key=lambda x : x[1], reverse=True)
    stats[dsname]['python_rrf_speed'].append( (time.time()-starttime) )
    mres3 = mres3[:5]
    stats[dsname]['python_rrf_correct'].append( iscorrectintopk( questionsolutions[dsname][qi], mres3 ) )
    if verbose :
      print('---- Python RRF')
      print('id | rrfscore | doctext')
      for r in mres3 :
        print(r[0],r[1],r[2])

    # bm25rrf
    if verbose : print('---- bm25rrf')
    starttime = time.time()
    mres4 = msq( 'SELECT * FROM bm25rrf(\''+q.replace("'","\'\'")+'\', \''+json.dumps( embeddedquestions[dsname][qi].tolist() )+'\', \''+tablename+'\', \''+idcolumnname+'\', \''+columnname+'\', \''+embeddingcolumnname+'\', \''+str(slimit)+'\', \''+algo +'\');', verbose )
    stats[dsname]['bm25rrf_speed'].append( (time.time()-starttime) )
    mres4 = mres4[:5]
    stats[dsname]['bm25rrf_correct'].append( iscorrectintopk( questionsolutions[dsname][qi], mres4 ) )

    # correct check after RRF
    mres = mres[:5]
    mres2 = mres2[:5]
    stats[dsname]['plpgsql_bm25_correct'].append( iscorrectintopk( questionsolutions[dsname][qi], mres ) )
    stats[dsname]['pgvector_correct'].append( iscorrectintopk( questionsolutions[dsname][qi], mres2 ) )

    #### End of questions loop

  # format and print stats
  stats2[dsname] = {}
  for statname in stats[dsname] :
    stats2[dsname][statname+' total'] = [ sum(stats[dsname][statname]) ]
  #
  for statname in stats2[dsname] :
    print( statname, stats2[dsname][statname][0] )
    stats['total'][statname] += stats2[dsname][statname][0]
  #
  if stats2[dsname]['python_rrf_correct total'][0] != stats2[dsname]['bm25rrf_correct total'][0] :
    stats['diffs'].append( {'dsname':dsname, 'python_rrf_correct total': stats2[dsname]['python_rrf_correct total'][0], 'bm25rrf_correct total': stats2[dsname]['bm25rrf_correct total'][0]} )

  #### End of datasets loop

# print results
print('\n-------- Total stats')
for k in stats['total'] :
  print( k, ':', stats['total'][k] )
print('\n-------- diffs')
print( stats['diffs'] )



---- msmarco ----------------------------------------------------

plpgsql_bm25_speed total 6.391732931137085
plpgsql_bm25_correct total 362
pgvector_speed total 4.242866516113281
pgvector_correct total 389
python_rrf_speed total 0.33017921447753906
python_rrf_correct total 384
bm25rrf_speed total 7.491535902023315
bm25rrf_correct total 383

---- robust04 ----------------------------------------------------

plpgsql_bm25_speed total 6.46416711807251
plpgsql_bm25_correct total 350
pgvector_speed total 4.516360282897949
pgvector_correct total 352
python_rrf_speed total 0.35675048828125
python_rrf_correct total 368
bm25rrf_speed total 8.27994441986084
bm25rrf_correct total 368

---- nq ----------------------------------------------------

plpgsql_bm25_speed total 8.010437488555908
plpgsql_bm25_correct total 375
pgvector_speed total 5.412384510040283
pgvector_correct total 385
python_rrf_speed total 0.39949822425842285
python_rrf_correct total 391
bm25rrf_speed total 9.633994340896606
bm2